# Neki napredniji aspekti Pythona

## Klase

Vidjeli smo da u Pythonu varijable nemaju tipove, ali zato _objekti_ imaju tipove. Tip objekta određuje kakve vrijednosti taj objekt može imati (u slučaju da je promjenjiv), i važnije, određuje što sve možemo s tim objektom raditi, odnosno kako ga možemo koristiti.

Realne brojeve možemo potencirati, ali ne možemo njima indeksirati liste. Cijelim brojevima možemo indeksirati liste, ali im ne možemo mijenjati vrijednost. Listama možemo mijenjati vrijednost, ali ih ne možemo pozivati. Funkcije možemo pozivati, ali ih ne možemo množiti. Liste možemo množiti cijelim brojevima, ali ne i realnim brojevima. Izuzetke možemo dizati, klase instancirati, module uvoziti, ključeve u rječnicima hashirati, po skupovima iterirati,... I tako dalje: tipovi i njihove podržane operacije čine izuzetno složenu mrežu međuovisnosti. Ako pokušamo upotrijebiti objekt na način koji njegov tip ne podržava, dobit ćemo `TypeError` (za razliku od situacije gdje tip dozvoljava operaciju ali ne na konkretnoj vrijednosti, kada dobijemo `ValueError` ili neki još specifičniji izuzetak.

In [34]:
3.14 * [3, 4]

TypeError: can't multiply sequence by non-int of type 'float'

U Pythonu možemo definirati i svoje klase, čiji objekti će onda imati točno ona ponašanja koja im propišemo. Na neki način, time smo stvorili novi tip. Najjednostavnije što klase mogu imati su atributi: varijable u klasnom prostoru imena.

In [1]:
class Klasa1:
    varijabla = 5
    
Klasa1.varijabla

5

Klasi možemo dodavati (i brisati, i mijenjati...) atribute po volji.

In [2]:
Klasa1.atribut = 7

del Klasa1.varijabla

Kad klasu instanciramo (sintaksa je ista kao za poziv funkcije, postfiksne zagrade), dobivamo objekt koji ima pristup svim varijablama klase.

In [3]:
instanca1 = Klasa1()
type(instanca1)

__main__.Klasa1

In [4]:
instanca1.atribut

7

Sve je dinamično: dodavanjem novog atributa klasi omogućujemo pristup tom atributu od svih objekata klase, čak i onih koji su stvoreni prije nego što je atribut dodan klasi.

In [5]:
Klasa1.novo = 9
instanca1.novo

9

Pored klasnih atributa (što bi donekle odgovaralo "statičkim varijablama"), instance mogu imati i vlastite (individualne) atribute.

In [6]:
instanca2 = Klasa1()
instanca1.id = 3
instanca2.id = 4
for objekt in instanca1, instanca2:
    print(objekt.id + objekt.novo)

12
13


Dakle, možemo reći da se instanciranjem na neki način povezuju atributni prostor imena instance i klase (koji pak nastaje izvršavanjem bloka koji počinje s `class`). Ipak, ta poveznica nije identiteta za sve tipove atributa.

Za početak, u klasnom prostoru imena mogu postojati funkcije.

In [7]:
class Klasa2:
    def funkcija():
        print('Ja sam funkcija')

Klasa2.funkcija()

Ja sam funkcija


... i kao što vidimo, ništa se ne mijenja kad im pristupamo iz klasnog prostora imena. No iz individualnog prostora imena, situacija je drugačija.

In [8]:
instanca = Klasa2()
instanca.funkcija()

TypeError: funkcija() takes 0 positional arguments but 1 was given

Vidimo da kad pozovemo `instanca.funkcija()`, Python zapravo poziva `Klasa2.funkcija` s jednim argumentom. Kojim to? Naravno, ako znate priču o `this` iz C++a, jasno je da se poziva `Klasa2.funkcija(instanca)`. I općenito, u normalnim okolnostima, `objekt.metoda(argumenti)` se izvršava kao `type(objekt).metoda(objekt, argumenti)`, odnosno objekt na kojem je metoda pozvana prenosi se kao prvi argument. Da bi se na to podsjetili, ljudi obično prvi parametar metode zovu `self`.

Primijetite da je (za razliku od `this`) `self` eksplicitan (mora se navesti). Kako nema deklaracijâ, Python ne može znati predstavlja li `x` lokalnu varijablu metode ili člansku varijablu objekta na kojem je metoda pozvana. Vjerojatno najčešća greška C++-programera u Pythonu je da pristupaju atributima objekta na kojem je metoda pozvana bez `self`. No i sâm C++ sve više ide u smjeru sintaksnog razlikovanja članskih i lokalnih varijabli: npr. vidio sam da prof. Jurak u zadaćama inzistira na imenovanju članskih varijabli imenima koja počinju s `m_` (recimo `m_x`). U tom kontekstu, možemo reći da Python inzistira na imenovanju članskih varijabli imenima koja počinju sa `self.` (recimo `self.x`). :-)

In [9]:
class Klasa3:
    def metoda(self, argument, poruka):
        print(poruka)
        return self.id + argument
    
instanca = Klasa3()
instanca.metoda(5, 'Zbroj je')

Zbroj je


AttributeError: 'Klasa3' object has no attribute 'id'

Vidimo da često moramo postavljati individualne atribute objekata koje konstruiramo na neke vrijednosti (i dobivamo grešku kad zaboravimo). To možemo raditi ručno:

In [10]:
instanca.id = 2
instanca.metoda(5, 'Zbroj je')

Zbroj je


7

... ali bilo bi daleko bolje kad bi se to automatski dogodilo konstrukcijom. C++ za to ima konstruktore, pa se u Pythonu analogna stvar često zove "konstruktor", iako je "inicijalizator" bolji naziv (jer `self` je već konstruiran, samo mu treba postaviti atribute). To je metoda imena `__init__`, koja se izvrši automatski nakon instanciranja klase (pod pretpostavkom da instanciranje stvarno vrati objekt te klase, što se događa u svim normalnim slučajevima).

In [11]:
class Klasa4:
    metoda = Klasa3.metoda  # klasni namespace je kao i svaki drugi :-)
    
    def __init__(self):
        print('Inicijalizacija')
        self.id = 0

instanca = Klasa4()
instanca.metoda(3, 'Metoda')

Inicijalizacija
Metoda


3

Naravno, kao i u C++u, inicijalizator može imati argumente, koji mu se prenose "pozivom" klase.

In [12]:
class Klasa5:
    metoda = Klasa3.metoda
    
    def __init__(self, id):
        self.id = id

instanca = Klasa5(6)
instanca.metoda(1, '6+1=')

6+1=


7

Osim `__init__`, postoji i hrpa (stotinjak) specijalnih "magičnih" metodâ koje omogućavaju sudjelovanje instanciranih objekata u raznim sintaksnim konstruktima, pokazat ćemo samo neke. (Puno više, iako ne sve, možete naći [ovdje](https://docs.python.org/3/reference/datamodel.html#basic-customization)).

In [13]:
class Vaga:
    def __init__(self, a, b):
        self.a, self.b = a, b
        
    def __str__(self):  # pretvorba u string, npr. za print
        return rf'\_{self.a}_/#\_{self.b}_/'
    
    def __bool__(self):  # pretvorba u bool, npr. za if
        return self.a == self.b
        
    def __add__(self, other):  # zbrajanje (operator+)
        return Vaga(self.a + other.a, self.b + other.b)
    
    def __invert__(self):  # operator~
        return Vaga(self.b, self.a)

v = Vaga(2, 1)
print(v, end='\n'*2)
v += ~v
print(v, end='\n'*2)
if v:
    print('U ravnoteži')

\_2_/#\_1_/

\_3_/#\_3_/

U ravnoteži


Ako nam klasa služi samo kao `struct`, odnosno za držanje većih količina imenovanih podataka na okupu, tako da sve instance imaju iste atribute, nespretno je pisati hrpu inicijalizacija.

In [ ]:
class Student:
    def __init__(self, ime, jmbag, ocjene, godina_rođenja):
        self.ime = ime
        self.jmbag = jmbag
        self.ocjene = ocjene
        self.godina_rođenja = godina_rođenja
        ...  # itd.

Tada možemo koristiti dekorator `@dataclass` iz modula `dataclasses`, gdje samo trebamo navesti ("deklarirati") atribute koje želimo da sve instance imaju. Iz toga automatski dobijemo odgovarajući inicijalizator, ali i lijep ispis, usporedbu po jednakosti, i još neke detalje (neke, kao što su hashiranje, uređaj i nepromjenjivost, možemo eksplicitno uključiti, pogledajte [dokumentaciju](https://docs.python.org/3/library/dataclasses.html)).

In [19]:
from dataclasses import dataclass

@dataclass
class Student:
    ime: str
    jmbag: str
    ocjene: list
    godina_rođenja: int
        
    def starost(self, rođendan=True):
        from datetime import datetime
        trenutna_godina = datetime.now().year
        godine = trenutna_godina - self.godina_rođenja
        if not rođendan: godine -= 1
        return godine
    
    def prosjek(self):
        import numpy as np
        return np.mean(self.ocjene)
    
    def __format__(self, frm):
        if frm == 'novine':
            naslov = type(self).__name__.lower()
            ime, prezime, *_ = self.ime.split()
            return f'{naslov} {ime[0]}.{prezime[0]}.({self.starost()})'
        else: return format(self.ime, frm)

Marko = Student('Marko Lukarić', '1191555000', [5, 4, 3, 5], 1998)
Marko

Student(ime='Marko Lukarić', jmbag='1191555000', ocjene=[5, 4, 3, 5], godina_rođenja=1998)

In [20]:
f'Pismo nam je poslao {Marko:novine}'

'Pismo nam je poslao student M.L.(22)'

In [18]:
f'Imam {6+3.2:+03.2} ovaca.'

'Imam +9.2 ovaca.'

Klase mogu nasljeđivati jedna drugu. Klase od kojih nasljeđujemo pišemo u zagradama pri deklaraciji klase. Recimo, uobičajeniji način da realiziramo klasu `Klasa4` s istom metodom `metoda` kao `Klasa3` bio bi:

In [ ]:
class Klasa4(Klasa3):
    def __init__(self):
        ...

(Moguća su i višestruka nasljeđivanja.) Ako želimo unutar neke metode pozvati metodu natklase (ili su-klase u slučaju suradničkog nasljeđivanja) umjesto na `self` pozivamo je na `super()`:

In [21]:
class Brucoš(Student):
    def prosjek(self):
        p = super().prosjek()
        return f'{p}, ali prosjek brucoša je vrlo varijabilan'

t = Brucoš('Tin Kavić', '1191000555', [4], 2002)
t.prosjek()

'4.0, ali prosjek brucoša je vrlo varijabilan'

## Kompliciraniji primjer: algoritmi na grafovima

In [22]:
class Graf:
    def __init__(self, **bridovi):
        from collections import defaultdict
        self.bridovi = defaultdict(int)
        self.vrhovi = set()
        for (početak, kraj), težina in bridovi.items():
            self.vrhovi |= {početak, kraj}
            self.bridovi[početak, kraj] = težina
    
    def minstablo(self):
        """Razapinjuće podstablo danog grafa, najmanje težine.
        
        Staza od vrha a do vrha b je konačni niz vrhova a,...,b u kojem su susjedni vrhovi povezani.
        U stazi se ne smiju ponavljati vrhovi, osim što prvi može biti jednak zadnjem.
        Ciklus je staza od a do a, duljine (kao konačni niz) barem 4. Stablo je graf bez ciklusa.
        Podgraf od G je graf čiji su bridovi neki bridovi od G. Podstablo je podgraf koji je stablo.
        Težina stabla je zbroj težinâ svih njegovih bridova.
        Podstablo od G je razapinjuće ako ima isti skup vrhova kao G.
        """
        # Kruskalov algoritam
        h = {vrh: 0 for vrh in self.vrhovi}
        def komponenta(vrh):
            početak = vrh
            while h[vrh] in self.vrhovi: vrh = h[vrh]
            kraj = vrh
            
            vrh = početak
            while h[vrh] in self.vrhovi: h[vrh], vrh = kraj, h[vrh]
            return kraj, h[kraj]
        
        stablo = set()
        još_dodati = len(self.vrhovi) - 1
        if not još_dodati: return stablo
        from operator import itemgetter
        for brid in sorted(self.bridovi, key=self.bridovi.get):
            (V1, h1), (V2, h2) = [komponenta(vrh) for vrh in brid]
            if V1 != V2:
                stablo.add(brid)
                još_dodati -= 1
                if not još_dodati: return stablo
                if h1 < h2: h[V1] = V2
                else: h[V2] = V1
                if h1 == h2: h[V1] += 1

In [23]:
G = Graf(AD=2, AE=1, DE=1, FC=1, FB=1, CB=2, EF=2)
G.minstablo()

{('A', 'E'), ('D', 'E'), ('E', 'F'), ('F', 'B'), ('F', 'C')}

## Izuzetci

Prilikom poziva raznih funkcija u Pythonu, mogu se dogoditi greške, nepredviđene okolnosti, ili jednostavno nešto što predstavlja odstupanje od uobičajene kontrole toka. Python u svim tim slučajevima *diže izuzetak*: funkcija signalizira svom pozivatelju da se dogodilo nešto izvan uobičajene "evo ti argumenti, pošalji mi povratnu vrijednost"-komunikacije. Pozivatelj može učiniti nešto po tom pitanju, nakon čega se kontrola toka (obično) vraća na uobičajenu razinu; ili to može zanemariti, u kom slučaju će *njegov* pozivatelj dobiti informaciju o izuzetku. Tako se neobrađeni izuzetak "diže" sve dok ga netko ne obradi -- ili do globalne razine, u kom slučaju ga Python prijavljuje korisniku kao poruku o grešci.

In [31]:
def prva_funkcija():
    print('početak prve funkcije')
    druga_funkcija()
    print('kraj prve funkcije')
def druga_funkcija():
    print('početak druge funkcije')
    1 / 0
    print('kraj druge funkcije')
prva_funkcija()

početak prve funkcije
početak druge funkcije


ZeroDivisionError: division by zero

Prateći strelice, vidimo da je prvo u 9. liniji pozvana `prva_funkcija()`, koja je u 3. liniji pozvala `druga_funkcija()`, koja je pak u 7. liniji pokušala podijeliti broj nulom i time dignula `ZeroDivisionError`. Kako taj izuzetak ni `druga_funkcija` ni `prva_funkcija` nisu obradile, prijavljen je korisniku kao greška. 

Pored ogromne i razgranate [hijerarhije izuzetaka](https://docs.python.org/3/library/exceptions.html#exception-hierarchy) možemo i proizvoditi svoje izuzetke. Stvaranje novih izuzetaka je jednostavno, sve što Python zahtijeva je da ih naslijedimo iz neke već postojeće klase izuzetaka (u korijenu je `BaseException`, ali najčešće možemo upotrijebiti nešto specijalnije, recimo `Exception` ili `RuntimeError`). Dobro je uvijek staviti _custom_ klasu izuzetaka što niže, jer će tako biti uhvaćena (i vjerojatno ispravno obrađena) u većem broju slučajeva.

In [25]:
class MojIzuzetak(RuntimeError):
    """Primjer proizvodnje vlastite klase izuzetaka."""

raise MojIzuzetak

MojIzuzetak: 

Vidimo da možemo dizati izuzetke naredbom `raise`. Ako ih želimo obraditi, koristimo naredbu `try`, oblika `try...except...except...else...finally...` (ne moramo navesti sve dijelove). 

* `try` označava osnovni blok koda u kojem se prate izuzetci. Dobro je da bude što kraći (najčešće jedna naredba), da ne bismo slučajno maskirali izuzetak koji ne želimo.
* `except Klasa as e` označava blok unutar kojeg se ime `e` odnosi na izuzetak klase `Klasa` ako je takav dignut za vrijeme izvršavanja osnovnog bloka. `as e` ne moramo napisati ako nas ne zanimaju konkretna instanca izuzetka, već samo njen tip.
* `else` (u ovom kontekstu) označava blok koda koji se izvršava ako je osnovni blok završio "uobičajeno" (bez izuzetaka). To nije isto kao dodavanje naredbi na kraj osnovnog bloka, jer se u ovom bloku izuzetci ne prate.
* `finally` označava blok koda koji se uvijek izvrši, bez obzira na to je li za vrijeme osnovnog bloka dignut ikakav izuzetak (bilo obrađen s `except` ili ne) ili nije. Upotrebljava se za vraćanje korištenih resursa u prvobitno stanje.

In [28]:
try:
    tekst = input('Broj koji treba kvadrirati: ')
    broj = int(tekst)
except ValueError:
    print(f'{tekst} nije broj')
except KeyboardInterrupt:
    print('Unos prekinut')
else:
    print(broj ** 2)

Broj koji treba kvadrirati: 2
4


## Generatori

Ako imamo neku kompliciranu kontrolu toka (petlje, grananja, izuzetci,...) _unutar_ koje se neki segmenti ponavljaju s manjim izmjenama, lako ih je izdvojiti u zasebnu funkciju (manje izmjene apstrahiramo kao parametre) koju onda pozovemo kad treba. No što ako je _vanjska_ struktura takva da je želimo izdvojiti u funkciju?

Čest slučaj su višestruke petlje s raznim uvjetima. Recimo, za zadani $n$, želimo da $x$ ide od $0$ do isključivo $n$, a $y$ je uvijek "blizu" $x$ (u smislu da je $|x-y|\le1$), ali tako da ne izađe iz raspona varijable $x$.

    for x in range(n):
        for delta in -1, 0, 1:
            y = x + delta
            if y in range(n):
                ... napravi nešto s x i y
    for x in range(m):
        for delta in -1, 0, 1:
            y = x + delta
            if y in range(m):
                ... napravi nešto drugo s x i y
    ...
    
Dakle, želimo "okvir" apstrahirati u zasebnu funkciju (s parametrom $n$), a unutarnje dijelove ostaviti u glavnom kodu:

    for x, y in susjedi_do(n): ... napravi nešto s x i y
    for x, y in susjedi_do(m): ... napravi nešto drugo s x i y

Jedno rješenje je svakako napraviti funkciju koja vraća listu parova. Tu listu možemo izgraditi i komprehenzijom ako je dovoljno jednostavna. No to predstavlja besmisleni utrošak memorije: za $n=10^6$, imat ćemo oko tri milijuna parova brojeva odjednom u memoriji, a zapravo nam trebaju samo jedan po jedan par.

Rješenje je upotreba _generatora_. Generatori su funkcije koje mogu dati nešto pozivatelju, ali ne kao povratnu vrijednost (završavajući time svoje izvršavanje), već neodređeni broj objekata jedan po jedan. Da bismo istaknuli razliku u semantici od običnih funkcija, umjesto `return` u generatorima koristimo `yield`.

In [29]:
def susjedi_do(n):
    for x in range(n):
        for delta in -1, 0, 1:
            y = x + delta
            if y in range(n):
                yield x, y

In [30]:
for x, y in susjedi_do(10**6):
    if x + y == 1234567: print(x, y)
print('-'*20)
for prvi, drugi in susjedi_do(100):
    if prvi**drugi == drugi**prvi + 1: print(prvi, drugi)

617283 617284
617284 617283
--------------------
1 0
2 1
3 2


Generatori (precizno, njihove povratne vrijednosti) su vrsta *iteratora*, objekata po kojima se može iterirati. Sučelje za rad s iteratorima predstavlja funkcija `next`, koja daje sljedeći element ili diže `StopIteration`.

In [31]:
g = susjedi_do(2)

In [32]:
next(g)

(0, 0)

In [33]:
next(g) + next(g), next(g)

((0, 1, 1, 0), (1, 1))

In [34]:
next(g)

StopIteration: 

Osim iteratora, mnogi drugi objekti (npr. spremnici) su *iterabilni*, u smislu da se funkcijom `iter` može dobiti iterator koji prolazi kroz njihove elemente. Razlika između iterabilnih objekata i iteratora je u tome što su iteratori "za jednokratnu upotrebu", odnosno služe za jedan prolaz kroz iterabilni objekt. Npr. gornji `g` je sada beskoristan, ne možemo ga "premotati" -- ali u slučaju spremnika, uvijek možemo funkcijom `iter` dobiti novi iterator kroz njih.

In [35]:
lista = [8, 7, 17, 3]

In [28]:
i1 = iter(lista)
next(i1), next(i1)

(8, 7)

In [29]:
i2 = iter(lista)
next(i2), next(i1), next(i2)

(8, 17, 7)

In [36]:
import itertools